<a href="https://colab.research.google.com/github/youngmin5068/ComputerVision/blob/main/MMdetection_video_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openmim
!mim install mmdet

In [2]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 31759, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 31759 (delta 105), reused 266 (delta 84), pack-reused 31420
Receiving objects: 100% (31759/31759), 40.93 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (22725/22725), done.


In [ ]:
!cd mmdetection; python setup.py install

In [ ]:
!pip uninstall mmdet
!pip install openmim
!mim install mmdet

In [2]:
!cd mmdetection; mkdir checkpoints

In [ ]:
!wget -O /content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [24]:
config_file = '/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = '/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

In [40]:
!wget -O /content/John_Wick_small.mp4 https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true

--2022-10-03 09:27:20--  https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/chulminkw/DLCV/raw/master/data/video/John_Wick_small.mp4 [following]
--2022-10-03 09:27:21--  https://github.com/chulminkw/DLCV/raw/master/data/video/John_Wick_small.mp4
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/John_Wick_small.mp4 [following]
--2022-10-03 09:27:21--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/John_Wick_small.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.

In [41]:
from mmdet.apis import init_detector, inference_detector
import mmcv

In [42]:
model = init_detector(config_file, checkpoint_file, device='cuda:0')

load checkpoint from local path: /content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


In [43]:
# https://github.com/open-mmlab/mmdetection/blob/master/demo/video_demo.py 대로 video detection 수행. 

import cv2

video_reader = mmcv.VideoReader('/content/John_Wick_small.mp4')
video_writer = None
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter('/content/John_Wick_small_out1.mp4', fourcc, video_reader.fps,(video_reader.width, video_reader.height))

for frame in mmcv.track_iter_progress(video_reader):
  result = inference_detector(model, frame)
  frame = model.show_result(frame, result, score_thr=0.4)

  video_writer.write(frame)

if video_writer:
        video_writer.release()

[                                                  ] 0/58, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/mmdet-2.25.2-py3.7.egg/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 58/58, 3.7 task/s, elapsed: 16s, ETA:     0s


In [45]:
# 0부터 순차적으로 클래스 매핑된 label 적용. 
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorbike',4:'aeroplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',10:'fire hydrant',
                        11:'stop sign',12:'parking meter',13:'bench',14:'bird',15:'cat',16:'dog',17:'horse',18:'sheep',19:'cow',20:'elephant',
                        21:'bear',22:'zebra',23:'giraffe',24:'backpack',25:'umbrella',26:'handbag',27:'tie',28:'suitcase',29:'frisbee',30:'skis',
                        31:'snowboard',32:'sports ball',33:'kite',34:'baseball bat',35:'baseball glove',36:'skateboard',37:'surfboard',38:'tennis racket',39:'bottle',40:'wine glass',
                        41:'cup',42:'fork',43:'knife',44:'spoon',45:'bowl',46:'banana',47:'apple',48:'sandwich',49:'orange',50:'broccoli',
                        51:'carrot',52:'hot dog',53:'pizza',54:'donut',55:'cake',56:'chair',57:'sofa',58:'pottedplant',59:'bed',60:'diningtable',
                        61:'toilet',62:'tvmonitor',63:'laptop',64:'mouse',65:'remote',66:'keyboard',67:'cell phone',68:'microwave',69:'oven',70:'toaster',
                        71:'sink',72:'refrigerator',73:'book',74:'clock',75:'vase',76:'scissors',77:'teddy bear',78:'hair drier',79:'toothbrush' }


In [47]:
def get_detected_img(model, img_array, score_threshold=0.3, is_print=True):
  draw_img = img_array.copy()
  bbox_color=(0,255,0)
  text_color=(0,0,255)

  results = inference_detector(model,img_array)

  for result_idx, result in enumerate(results):
    
    if len(result)==0:
      continue
    
    result_filtered = result[np.where(result[:,4]>score_threshold)]

    for i in range(len(result_filtered)):
      left = int(result_filtered[i,0])
      top = int(result_filtered[i,1])
      right = int(result_filtered[i,2])
      bottom = int(result_filtered[i,3])
      caption="{}: {:.4f}".format(labels_to_names_seq[result_idx], result_filtered[i,4])
      cv2.rectangle(draw_img, (left,top),(right,bottom),color=bbox_color, thickness=2)
      cv2.putText(draw_img, caption, (int(left),int(top)-7), cv2.FONT_HERSHEY_SIMPLEX,0.37,text_color,1)
      if is_print:
        print(caption)
  return draw_img

In [48]:
import time

def do_detected_video(model, input_path, output_path, score_threshold, do_print=True):
    
    cap = cv2.VideoCapture(input_path)

    codec = cv2.VideoWriter_fourcc(*'XVID')

    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_fps = cap.get(cv2.CAP_PROP_FPS)

    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size) 

    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)
    btime = time.time()
    while True:
        hasFrame, img_frame = cap.read()
        if not hasFrame:
            print('더 이상 처리할 frame이 없습니다.')
            break
        stime = time.time()
        img_frame = get_detected_img(model, img_frame,  score_threshold=score_threshold, is_print=False)
        if do_print:
          print('frame별 detection 수행 시간:', round(time.time() - stime, 4))
        vid_writer.write(img_frame)
    # end of while loop

    vid_writer.release()
    cap.release()

    print('최종 detection 완료 수행 시간:', round(time.time() - btime, 4))

In [49]:
do_detected_video(model, '/content/John_Wick_small.mp4', '/content/John_Wick_small_out2.mp4', score_threshold=0.7, do_print=True)

총 Frame 갯수: 58
frame별 detection 수행 시간: 0.1592
frame별 detection 수행 시간: 0.1225
frame별 detection 수행 시간: 0.1197
frame별 detection 수행 시간: 0.108
frame별 detection 수행 시간: 0.1048
frame별 detection 수행 시간: 0.108
frame별 detection 수행 시간: 0.1077
frame별 detection 수행 시간: 0.1066
frame별 detection 수행 시간: 0.1051
frame별 detection 수행 시간: 0.1066
frame별 detection 수행 시간: 0.1063
frame별 detection 수행 시간: 0.1103
frame별 detection 수행 시간: 0.112
frame별 detection 수행 시간: 0.1036
frame별 detection 수행 시간: 0.1086
frame별 detection 수행 시간: 0.1109
frame별 detection 수행 시간: 0.106
frame별 detection 수행 시간: 0.1093
frame별 detection 수행 시간: 0.1023
frame별 detection 수행 시간: 0.1079
frame별 detection 수행 시간: 0.1058
frame별 detection 수행 시간: 0.1067
frame별 detection 수행 시간: 0.1077
frame별 detection 수행 시간: 0.1058
frame별 detection 수행 시간: 0.1052
frame별 detection 수행 시간: 0.1065
frame별 detection 수행 시간: 0.1074
frame별 detection 수행 시간: 0.1071
frame별 detection 수행 시간: 0.1051
frame별 detection 수행 시간: 0.1079
frame별 detection 수행 시간: 0.1055
frame별 detection 수행 시간: 0.10